In [1]:
!{__import__('sys').executable} -m pip install --quiet --upgrade -r requirements.txt

In [2]:
session_storage = 'downloads'

import os
os.makedirs(session_storage, exist_ok=True)

In [3]:
import fsspec

fs_dataset = fsspec.filesystem(
    'simplecache', 
    target_protocol='https', 
    cache_storage=session_storage,
)

In [4]:

# https://www.diva-portal.org/smash/get/diva2:1571821/FULLTEXT02

# https://keras.io/examples/vision/attention_mil_classification/
# https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [5]:
#from urllib.parse import urlparse
#import os
#os.path.split(
#    urlparse('/Movie Scripts/Reservoir Dogs Script.html').path
#)
#len(list(filter(lambda s: s.endswith('.html'), get_movie_links())))

In [6]:
import requests
import lxml
import lxml.html

import os
import urllib


def get_all_hrefs(url):
    return map(
        lambda s: urllib.parse.urlparse(s).path,
        lxml.html.fromstring(
            requests.get(url).text
        ).xpath('//a/@href')
    )


import joblib

def get_script(script_link):
    res = ''
    for el in lxml.html.fromstring(
        requests.get('https://imsdb.com/' + script_link).text
    ).xpath('//td[@class="scrtext"]/pre'):    
        res += ''.join(el.itertext())
    return res    

def get_movie_links():
    return set(
        filter(
            lambda u: os.path.dirname(u) == '/Movie Scripts',
            get_all_hrefs('https://imsdb.com/all-scripts.html')
        )
    )

def get_script_links(movie_link):
    return set(
        filter(
            lambda u: os.path.dirname(u) == '/scripts',            
            get_all_hrefs('https://imsdb.com/' + movie_link)
        )
    )
    
def get_movie_name(movie_link):
    name, _ = os.path.splitext(
        os.path.basename(movie_link)
    )
    return name
    
def get_movie_script_links():
    def _f(movie_link):
        return (
            get_movie_name(movie_link),
            get_script_links(movie_link)
        )
    return joblib.Parallel(n_jobs=-1)(
        joblib.delayed(_f)(movie_link) for movie_link in get_movie_links()
    )

def get_scripts(movie_script_links):
    def _f(movie_name, script_link):
        return (
            movie_name,
            list(map(
                get_script,
                script_link
            ))
        )

    return joblib.Parallel(n_jobs=-1)(
        joblib.delayed(_f)(movie_name, script_link) 
            for movie_name, script_link in movie_script_links
    )


In [7]:
import pandas as pd

df = pd.DataFrame(
    get_scripts(
        get_movie_script_links()
    ),
    columns=['script_name', 'script_text']
)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   script_name  1209 non-null   object
 1   script_text  1209 non-null   object
dtypes: object(2)
memory usage: 19.0+ KB


In [9]:
df.to_pickle('./dataset.pkl', compression='gzip')

In [10]:
pd.read_pickle('./dataset.pkl', compression='gzip')

,script_name,script_text
0,No Country for Old Men Script,[\r\n\r\n\r\n \r\n NO...
1,Strangers on a Train Script,[ STRANGERS ON A TRAIN\r\n\r...
2,"Sex, Lies and Videotape Script",[\r\n\r\n<b><!--\r\n</b>if (window!= top)\r\nt...
3,Absolute Power Script,[ABSOLUTE POWER\r\n\r\n Written by\r\n ...
4,"World is not Enough, The Script",[\t\t\r\n\r\n T H E W O R L D I S ...
...,...,...
1204,Shivers Script,[\r\n\r\n\r\n ...
1205,American Psycho Script,[\r\n\r\n\r\n<b><!--\r\n</b>if (window!= top)\...
1206,Five Easy Pieces Script,[\r\n\r\n\r\n<b><!--\r\n</b>if (window!= top)\...
1207,Saving Mr. Banks Script,[\r\n\r\n\r\n ...


In [11]:
#print(df.iloc[1]['script_text'][0])

In [12]:
import pandas as pd

In [13]:
str('Ace-Ventura-Pet-Detective.html').replace('.html', '')

'Ace-Ventura-Pet-Detective'

In [14]:
def get_movie_details(movie_links):
    pass

